## Summer Events in Seattle vs. Seattle AirBnB Market

1. Which major Seattle events seemed to have had an impact on Seattle AirBNB Market?
2. What locations appear to be most effected by event attendance?
4. Were fees influenced by the events

In [1]:
from datetime import timedelta
import pandas as pd
import numpy as np

from scripts.utility import *

# set data paths
raw_path = "data/raw/"
preproc_path = "data/intermediate/preproc/"

%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# load data
listings_df = pd.read_csv(raw_path + "listings.csv")
calendar_df = pd.read_csv(raw_path + "calendar.csv")
reviews_df = pd.read_csv(raw_path + "reviews.csv")

<IPython.core.display.Javascript object>

### Basic summary of data

In [3]:
print(calendar_df["listing_id"].count())
print(listings_df["id"].count())
print(reviews_df["id"].count())

2641505
7237
398157


<IPython.core.display.Javascript object>

In [4]:
# check percentage of null values
(len(calendar_df) - calendar_df.count()).div(len(calendar_df) * 100)

listing_id        0.0
date              0.0
available         0.0
price             0.0
adjusted_price    0.0
minimum_nights    0.0
maximum_nights    0.0
dtype: float64

<IPython.core.display.Javascript object>

There are 3,818 unique listings in the dataset to which 1,395,570 stays are attributed througout 2016. Prices are only given for days in which the airbnb was available. 

Availability is expressed as a state where the airbnb is blocked out, not necessarily booked. So this metric cannot be taken as occupancy metric. For this, historical review frequency was used as a rough analog for occupancy.

### Preprocessing

In [5]:
calendar_df.count()

listing_id        2641505
date              2641505
available         2641505
price             2641505
adjusted_price    2641505
minimum_nights    2641505
maximum_nights    2641505
dtype: int64

<IPython.core.display.Javascript object>

In [6]:
# drop duplicate records
# calendar_df.drop_duplicates()
# reviews_df.drop_duplicates()
# listings_df.drop_duplicates()

<IPython.core.display.Javascript object>

In [7]:
# convert price to clean floating field
calendar_df["price"] = format_dollar_field(calendar_df["price"])
calendar_df["adjusted_price"] = format_dollar_field(calendar_df["adjusted_price"])
listings_df["security_deposit"] = format_dollar_field(listings_df["security_deposit"])
listings_df["cleaning_fee"] = format_dollar_field(listings_df["cleaning_fee"])

# create various timeframe groupings for calendar
calendar_df["date"] = pd.to_datetime(calendar_df["date"])
calendar_df["week"] = calendar_df["date"].map(lambda date: date.isocalendar()[1])
calendar_df["month"] = calendar_df["date"].map(lambda date: date.month)

<IPython.core.display.Javascript object>

In [8]:
# create various timeframe groupings for reviews
reviews_df["date"] = pd.to_datetime(reviews_df["date"])
reviews_df["week"] = reviews_df["date"].map(lambda date: date.isocalendar()[1])
reviews_df["month"] = reviews_df["date"].map(lambda date: date.month)
reviews_df["year"] = reviews_df["date"].map(lambda date: str(date.year))

<IPython.core.display.Javascript object>

In [9]:
listings_df.groupby(["neighbourhood_group_cleansed", "neighbourhood_cleansed"]).count()

id  listing_url  \
neighbourhood_group_cleansed neighbourhood_cleansed                     
Ballard                      Adams                   115          115   
                             Loyal Heights            74           74   
                             Sunset Hill              46           46   
                             West Woodland            88           88   
                             Whittier Heights         58           58   
...                                                  ...          ...   
West Seattle                 Fauntleroy               28           28   
                             Gatewood                 46           46   
                             Genesee                  52           52   
                             North Admiral            99           99   
                             Seaview                  32           32   

                                                     scrape_id  last_scraped  \
neighbourhood_group_cleansed neighbourhood_cleansed                            
Ballard                      Adams                         115           115   
                             Loyal Heights                  74            74   
                             Sunset Hill                    46            46   
                             West Woodland                  88            88   
                             Whittier Heights               58            58   
...                                                        ...           ...   
West Seattle                 Fauntleroy                     28            28   
                             Gatewood                       46            46   
                             Genesee                        52            52   
                             North Admiral                  99            99   
                             Seaview                        32            32   

                                                     name  summary  space  \
neighbourhood_group_cleansed neighbourhood_cleansed                         
Ballard                      Adams                    115      115    104   
                             Loyal Heights             74       74     66   
                             Sunset Hill               46       46     43   
                             West Woodland             88       88     76   
                             Whittier Heights          58       55     51   
...                                                   ...      ...    ...   
West Seattle                 Fauntleroy                28       28     25   
                             Gatewood                  46       46     43   
                             Genesee                   52       52     45   
                             North Admiral             99       98     85   
                             Seaview                   32       32     27   

                                                     description  \
neighbourhood_group_cleansed neighbourhood_cleansed                
Ballard                      Adams                           115   
                             Loyal Heights                    74   
                             Sunset Hill                      46   
                             West Woodland                    88   
                             Whittier Heights                 58   
...                                                          ...   
West Seattle                 Fauntleroy                       28   
                             Gatewood                         46   
                             Genesee                          52   
                             North Admiral                    99   
                             Seaview                          32   

                                                     experiences_offered  \
neighbourhood_group_cleansed neighbourhood_cleansed                        
Ballard                 

<IPython.core.display.Javascript object>

In [10]:
# get a location-based variable with with transit and review count
listings_trimmed_df = listings_df[
    [
        "id",
        "neighbourhood_group_cleansed",
        "latitude",
        "longitude",
        "transit",
        "zipcode",
        "number_of_reviews",
        "security_deposit",
        "cleaning_fee",
    ]
]
location_df = calendar_df.join(listings_trimmed_df, lsuffix="listing_id", rsuffix="id")

# rename to correct spelling :)
location_df = location_df.rename(
    columns={"neighbourhood_group_cleansed": "neighborhood"}
)
location_df["id"] = location_df["id"].astype(str).str.replace(".0", "")

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,week,month,id,neighborhood,latitude,longitude,transit,zipcode,number_of_reviews,security_deposit,cleaning_fee
0,2318,2020-04-23,f,375.0,375.0,7,1125,17,4,2318,Central Area,47.61082,-122.29082,NaN,98122,32.0,500.0,250.0
1,261912,2020-04-23,f,200.0,200.0,4,14,17,4,66,Other neighborhoods,47.65411,-122.33761,NaN,98103,150.0,200.0,40.0
2,261912,2020-04-24,f,200.0,200.0,4,14,17,4,9419,Other neighborhoods,47.55017,-122.31937,Car 2 go is in this neigborhood Bus is across ...,98108,148.0,100.0,20.0
3,261912,2020-04-25,f,200.0,200.0,4,14,17,4,9531,West Seattle,47.55539,-122.38474,The Metro bus transit center and downtown wate...,98136,40.0,300.0,120.0
4,261912,2020-04-26,f,200.0,200.0,4,14,17,4,9534,West Seattle,47.55624,-122.38598,The local Metro bus transit center is four min...,98136,49.0,400.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641500,42841139,2021-04-18,f,293.0,293.0,2,1125,15,4,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2641501,42841139,2021-04-19,f,293.0,293.0,2,1125,16,4,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2641502,42841139,2021-04-20,f,293.0,293.0,2,1125,16,4,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2641503,42841139,2021-04-21,f,293.0,293.0,2,1125,16,4,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [11]:
# save intermediate data
calendar_df.to_csv(preproc_path + "calendar.csv", index=False)
reviews_df.to_csv(preproc_path + "reviews.csv", index=False)
location_df.to_csv(preproc_path + "location.csv", index=False)

<IPython.core.display.Javascript object>